## Default Model

In [1]:
#pip install scikit-learn==0.23.2

In [2]:
#import library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import metrics

In [3]:
#Columns to be considered

num_col = {'1st Flr SF',
 'Bsmt Cond',
 'Bsmt Exposure',
 'Bsmt Qual',
 'BsmtFin SF 1',
 'Exter Cond',
 'Exter Qual',
 'Fireplaces',
 'Full Bath',
 'Garage Cars',
 'Garage Cond',
 'Garage Qual',
 'Gr Liv Area',
 'Heating QC',
 'Kitchen Qual',
 'Mas Vnr Area',
 'Open Porch SF',
 'Overall Qual',
 'Total Bsmt SF',
 'Total_area',
 'Wood Deck SF',
 'age_since_built',
 'lot_frontage_imputed'}

cat_to_num = {'Exter Qual', 'Exter Cond', 'Bsmt Qual', 'Bsmt Cond',
    'Bsmt Exposure', 'Heating QC',
    'Kitchen Qual', 'Garage Qual', 'Garage Cond'}
     


cat_col = {'Bldg Type',
 'Foundation',
 'Street',
 'Central Air',
 'Electrical',
 'MS Zoning',
 'Land Contour',
 'Mas Vnr Type',
 'Heating',
 'Paved Drive'
          }

In [4]:
def cleaning_data(df):
    
    # 1. Drop most missing columns 
    df = df.drop(
    columns=['Pool QC', 'Misc Feature', 'Alley', 'Fence', 'Fireplace Qu'])

    # 2. Impute lot frontage
    from sklearn.linear_model import LinearRegression
    X_lot = df[df['Lot Frontage'] > 0][['1st Flr SF','Lot Area']]
    y_lot = df[df['Lot Frontage'] > 0]['Lot Frontage']
    lot_model = LinearRegression()
    lot_model.fit(X_lot,y_lot)
    
    lot_frontage_imputed = []
    for i in range(len(df)):
        if df.loc[i,'Lot Frontage'] > 0:
            lot_frontage_imputed.append(df.loc[i,'Lot Frontage'])
        else:
            lot_frontage_imputed.append(lot_model.predict([df.loc[i,['1st Flr SF','Lot Area']]])[0])

    df['lot_frontage_imputed'] = lot_frontage_imputed
    df = df.drop(columns = ['Lot Frontage'])
    
    #Garage Imputation

    df['Garage Yr Blt'] = df['Garage Yr Blt'].map(lambda x: x if x > 0 else 0)
    df['Garage Finish'] = df['Garage Finish'].map(lambda x: x if type(x) == str else 'NO')
    df['Garage Qual'] = df['Garage Qual'].map(lambda x: x if type(x) == str else 'NO')
    df['Garage Cond'] = df['Garage Cond'].map(lambda x: x if type(x) == str else 'NO')
    df['Garage Type'] = df['Garage Type'].map(lambda x: x if type(x) == str else 'NO')
    
    #Basement imputation
    df['Bsmt Exposure'] = df['Bsmt Exposure'].map(lambda x: x if type(x) == str else 'NO')
    df['BsmtFin Type 1'] = df['BsmtFin Type 1'].map(lambda x: x if type(x) == str else 'NO')
    df['BsmtFin Type 2'] = df['BsmtFin Type 2'].map(lambda x: x if type(x) == str else 'NO')
    df['Bsmt Cond'] = df['Bsmt Cond'].map(lambda x: x if type(x) == str else 'NO')
    df['Bsmt Qual'] = df['Bsmt Qual'].map(lambda x: x if type(x) == str else 'NO')
    
    #Mansory type and area
    df['Mas Vnr Type'] = df['Mas Vnr Type'].map(lambda x: x if type(x) == str else 'NO')
    df['Mas Vnr Area'] = df['Mas Vnr Area'].map(lambda x: x if x > 0 else 0)
    
    #Electrical Imputation
    df['Electrical'] = df['Electrical'].map(lambda x: x if type(x) == str else df['Electrical'].mode()[0])
    #Drop other observation containing Null
    df = df.dropna()
    
    return df

#Feature Engieering Function
def feat_eng(df):
    df['age_since_built'] = df['Yr Sold'] - df['Year Built']
    df['Total_area'] = df['Total Bsmt SF'] + df['1st Flr SF'] + df['2nd Flr SF']
    return df
#Converted OrdiNOl Categorical columns to Numerical 
def cat2num(df):
    for col in cat_to_num:
        df[col] = df[col].map(to_num)
    return df

#Conversion function
def to_num(val):
    if val == 'Ex':
        return 5
    elif val == 'Gd':
        return 4
    elif val == 'TA' or val == 'Av':
        return 3
    elif val == 'Fa' or val == 'Mn':
        return 2
    elif val == 'Po' or val == 'No':
        return 1
    elif val == 'NO':
        return 0



In [5]:
def prep(df,train_df):
    """
    df = DataFrame to be transformed
    train_df = cleaned training set
    """
    encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
    encoder.fit(train_df[cat_col])
    df_encode = pd.DataFrame(encoder.transform(df[cat_col]),
                             columns=encoder.get_feature_names(df[cat_col].columns),index=df.index)
    
    train_df_encode = pd.DataFrame(encoder.transform(train_df[cat_col]),
                             columns=encoder.get_feature_names(train_df[cat_col].columns),index=train_df.index)
    

    X_new = pd.concat([df[num_col],df_encode],axis=1)
    
    X_train = pd.concat([train_df[num_col],train_df_encode],axis=1)
    
    ss = StandardScaler()
    ss.fit(X_train)
    
    X_new_scaled = ss.transform(X_new)

    return X_new_scaled

Preprocessing order
1. Cleaning
1. Feature engineering
1. cat_to_num
1. OneHotEncoding

In [6]:
train = pd.read_csv('../datasets/train.csv')
train = cleaning_data(train)
train = feat_eng(train)
train = cat2num(train)
X_train = prep(train,train)
y_train = train['SalePrice']

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Pleas

In [7]:
test = pd.read_csv('../datasets/test.csv')
test = cleaning_data(test)
test = feat_eng(test)
test = cat2num(test)
X_test = prep(test,train)

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Pleas

## Model Training

### Instantiate

In [8]:
#MLR
lr = LinearRegression()

#Ridge LR
r_alphas = np.logspace(0, 5, 100)
ridge_cv = RidgeCV(alphas=r_alphas)

#Lasso LR
l_alphas = np.arange(0.001, 1, 0.0025)
lasso_cv = LassoCV(alphas=l_alphas)

#Elasticnet LR
l1_ratio = np.linspace(0, 1, 21)
enet_cv = ElasticNetCV(alphas=l_alphas, l1_ratio=l1_ratio)

### Training

In [9]:
#MLR Fitting
lr.fit(X_train,y_train)

#RidgeLR Fitting
ridge_cv.fit(X_train,y_train)

#LassoLR Fitting
lasso_cv.fit(X_train,y_train)

#ElasticNetLR Fitting
enet_cv.fit(X_train,y_train)

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26219293570.63965, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15387303865.999756, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 56951924125.56006, tolerance: 1040645604.631909
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1350645874754.475, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1349617630957.8418, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: U

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1310870999260.1318, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1309804054273.478, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: U

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1267311697617.4526, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1266198445628.116, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: U

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1221761844129.355, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1220594832282.0056, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: U

...

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1197488386.743164, tolerance: 1000280150.7726442
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19277328086.57251, tolerance: 1000280150.7726442
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 193538386724.79065, tolerance: 1000280150.7726442
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1297768387.6740723, tolerance: 1040645604.631909
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5404150326.101074, tolerance: 1040645604.631909
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22695020118.41382, tolerance: 1040645604.631909
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1838781865.00708, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6124883728.761475, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26295470299.974365, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1330200437.7529297, tolerance: 1000280150.7726442
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3252913484.829834, tolerance: 1000280150.7726442
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8246727567.931152, tolerance: 1000280150.7726442
  model = cd_fast.enet_coordinate_descent_gram(
/Users/wirachlee/opt/anaconda3/lib/python3

ElasticNetCV(alphas=array([0.001 , 0.0035, 0.006 , 0.0085, 0.011 , 0.0135, 0.016 , 0.0185,
       0.021 , 0.0235, 0.026 , 0.0285, 0.031 , 0.0335, 0.036 , 0.0385,
       0.041 , 0.0435, 0.046 , 0.0485, 0.051 , 0.0535, 0.056 , 0.0585,
       0.061 , 0.0635, 0.066 , 0.0685, 0.071 , 0.0735, 0.076 , 0.0785,
       0.081 , 0.0835, 0.086 , 0.0885, 0.091 , 0.0935, 0.096 , 0.0985,
       0.101 , 0.1035, 0.106 , 0.1085, 0.111 , 0.1135, 0.116 , 0.1185,
       0.121 , 0.123...
       0.901 , 0.9035, 0.906 , 0.9085, 0.911 , 0.9135, 0.916 , 0.9185,
       0.921 , 0.9235, 0.926 , 0.9285, 0.931 , 0.9335, 0.936 , 0.9385,
       0.941 , 0.9435, 0.946 , 0.9485, 0.951 , 0.9535, 0.956 , 0.9585,
       0.961 , 0.9635, 0.966 , 0.9685, 0.971 , 0.9735, 0.976 , 0.9785,
       0.981 , 0.9835, 0.986 , 0.9885, 0.991 , 0.9935, 0.996 , 0.9985]),
             l1_ratio=array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]))

In [10]:
#RMSE on training set
print('Linear Regression')
print(metrics.mean_squared_error(y_train,lr.predict(X_train),squared = False),'\n')
print('Ridge Regression')
print(metrics.mean_squared_error(y_train,ridge_cv.predict(X_train),squared = False),'\n')
print('Lasso Regression')
print(metrics.mean_squared_error(y_train,lasso_cv.predict(X_train),squared = False),'\n')
print('ElasticNet Regression')
print(metrics.mean_squared_error(y_train,enet_cv.predict(X_train),squared = False),'\n')

Linear Regression
30873.964416766394 

Ridge Regression
31040.460569690305 

Lasso Regression
30860.248430885906 

ElasticNet Regression
31064.262993310396 



### Cross Validation

In [11]:
ridge = Ridge(alpha=ridge_cv.alpha_)
lasso = Lasso(alpha=lasso_cv.alpha_)
enet = ElasticNet(alpha=enet_cv.alpha_, l1_ratio=enet_cv.l1_ratio_)

#Cross Validation RMSE
lr_cv_score = cross_val_score(
    lr, X_train, y_train, scoring='neg_root_mean_squared_error',
    cv=5).mean() * -1
ridge_cv_score = cross_val_score(
    ridge, X_train, y_train, scoring='neg_root_mean_squared_error',
    cv=5).mean() * -1
lasso_cv_score = cross_val_score(
    lasso, X_train, y_train, scoring='neg_root_mean_squared_error',
    cv=5).mean() * -1
enet_cv_score = cross_val_score(
    enet, X_train, y_train, scoring='neg_root_mean_squared_error',
    cv=5).mean() * -1

/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26219079145.635986, tolerance: 1026430467.7392805
  model = cd_fast.enet_coordinate_descent(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 56951141008.329346, tolerance: 1040645604.631909
  model = cd_fast.enet_coordinate_descent(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 31775430743.79883, tolerance: 1021922883.3405653
  model = cd_fast.enet_coordinate_descent(
/Users/wirachlee/opt/anaconda3/lib/python3.8/site-package

In [12]:
#Cross Validation RMSE
print('Linear Regression')
print(lr_cv_score,'\n')
print('Ridge Regression')
print(ridge_cv_score,'\n')
print('Lasso Regression')
print(lasso_cv_score,'\n')
print('ElasticNet Regression')
print(enet_cv_score,'\n')

Linear Regression
1.1209165591072403e+17 

Ridge Regression
32141.4447086 

Lasso Regression
32422.753903575107 

ElasticNet Regression
32136.14155177553 



the Best model ranked by cross validation score is ElasticNet Regression

### Making prediction and submit the result!

In [13]:
enet.fit(X_train,y_train)
Baseline_prediction = enet.predict(X_test)
Baseline_submission = pd.DataFrame(Baseline_prediction,
                                   index=test.Id,
                                   columns=['SalePrice'])
Baseline_submission.to_csv('../Submission/Baseline_submission.csv')